# Constants

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # only relevant to my own environment

I assume that you have downloaded MIDI dataset and uncompssed into `./midi_dataset/groove/` folder  
https://magenta.tensorflow.org/datasets/groove

In [2]:
DRUM_CLASSES = [
   'Kick',
   'Snare',
   'Hi-hat closed',
   'Hi-hat open',
   'Tom low',
   'Tom mid', 
   'Tom high',
   'Clap',
   'Rim' 
]
MIDI_DRUM_MAP = {
   36: 0,
   35: 0,
   38: 1,
   27: 1,
   28: 1,
   31: 1,
   32: 1,
   33: 1,
   34: 1,
   37: 1,
   39: 1,
   40: 1,
   56: 1,
   65: 1,
   66: 1,
   75: 1,
   85: 1,
   42: 2,
   44: 2,
   54: 2,
   68: 2,
   69: 2,
   70: 2,
   71: 2,
   73: 2,
   78: 2,
   80: 2,
   46: 3,
   67: 3,
   72: 3,
   74: 3,
   79: 3,
   81: 3,
   45: 4,
   29: 4,
   41: 4,
   61: 4,
   64: 4,
   84: 4,
   48: 5,
   47: 5,
   60: 5,
   63: 5,
   77: 5,
   86: 5,
   87: 5,
   50: 6,
   30: 6,
   43: 6,
   62: 6,
   76: 6,
   83: 6,
   49: 7,
   55: 7,
   57: 7,
   58: 7,
   51: 8,
   52: 8,
   53: 8,
   59: 8,
   82: 8
}

resolution  = 4 # separate quater into 4  = 16 notes per bar

nb_bars = 2 

len_seq = resolution * 4 * nb_bars # length of drumloops in training data - 2 bars
    
nb_notes = len(DRUM_CLASSES) # number of possible MIDI notes  - max_drum_note - min_drum_note


# Training

In [3]:
import numpy as np

matrices_drums = np.load("./tmp/matrices_drum_gm_electronic.npz")['drum_data']
matrices_genres = np.load("./tmp/matrices_drum_gm_electronic.npz")['genre_ids']
GENRES_ALL = np.load("./tmp/matrices_drum_gm_electronic.npz")['genres']

print(GENRES_ALL)
print(matrices_genres.shape)

# Filter genres 
GENRES_USED = [u'Old Skool', u'DnB', u'Jungle', u'House', u'Breakbeat', u'Garage', u'Techno']
GENRES_ID_USED = [i for i, genre in enumerate(GENRES_ALL) if genre in GENRES_USED]
GENRES = [genre for i, genre in enumerate(GENRES_ALL) if genre in GENRES_USED]

print(GENRES_ID_USED)
print(GENRES)

NB_GENRES = len(GENRES)
print("%d genres we have" % NB_GENRES)

_drums = []
_genres = []

for genre_id, drum in zip(matrices_genres, matrices_drums):
    if genre_id in GENRES_ID_USED:
        _drums.append(drum)
        genre_id = GENRES_ID_USED.index(genre_id)
        _genres.append(genre_id)

matrices_drums = np.array(_drums)
print(matrices_drums.shape)
matrices_genres = np.array(_genres)
print(matrices_genres.shape)

[u'Old Skool' u'Trance' u'DnB' u'Basic' u'Downtempo' u'Jungle' u'House'
 u'Breakbeat' u'Garage' u'Techno']
(49003,)
[0, 2, 5, 6, 7, 8, 9]
[u'Old Skool', u'DnB', u'Jungle', u'House', u'Breakbeat', u'Garage', u'Techno']
7 genres we have
(37950, 32, 9)
(37950,)


## Discriminator

In [4]:
import numpy as np
from keras.layers import Input, Dense, Flatten, Dropout, Reshape, LSTM, Bidirectional
from keras.layers import Embedding, Concatenate
from keras.layers.convolutional import Conv2D

from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import RMSprop, Adam
import keras.backend as K
from keras.models import Model, Sequential

batch_size = 16
n_z = 4
dropout_rate = 0.30

# encoder
drum_input = Input(shape=(len_seq, nb_notes), name='drum_input')  # tensorflow order

# # Conditioning label:
# # Integer 0-NB_GENRES denoting the genre G should generate
# label_input = Input(shape=(1,), dtype='int32')

# # Embedding layer:
# # Turns labels into dense vectors of size z_dim
# # Produces 3D tensor with shape: (batch_size, 1, 28*28*1)
# label_embedding = Embedding(NB_GENRES, np.prod((len_seq, nb_notes)), input_length=1)(label_input)

# # Flatten the embedding 3D tensor into 2D tensor with shape:(batch_size, 28*28*1)
# label_embedding = Flatten()(label_embedding)

# # Reshape label embeddings to have same dimensions as input images
# label_embedding = Reshape((len_seq, nb_notes))(label_embedding)

# # Concatenate images with corresponding label embeddings
# concatenated = Concatenate(axis=-1)([drum_input, label_embedding])


x = Bidirectional(LSTM(64, return_sequences=True, activation='tanh'))(drum_input) 
x = Bidirectional(LSTM(128, return_sequences=True, activation='tanh'))(x)
x = Bidirectional(LSTM(16, return_sequences=True, activation='tanh'))(x)
x1 = Reshape((1024,))(x)

x = Dense(512)(x1)
x = LeakyReLU(alpha=0.01)(x)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.01)(x)
output = Dense(1, activation='sigmoid', name='gan_output')(x)

Using TensorFlow backend.
W0826 14:42:39.566085 140360265770368 deprecation_wrapper.py:119] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 14:42:39.580029 140360265770368 deprecation_wrapper.py:119] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 14:42:39.583058 140360265770368 deprecation_wrapper.py:119] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [5]:
from keras.layers import Softmax

# genre classifier
# x = Bidirectional(LSTM(64, return_sequences=True, activation='tanh'))(drum_input) 
# x = Bidirectional(LSTM(128, return_sequences=True, activation='tanh'))(x)
# x = Bidirectional(LSTM(16, return_sequences=True, activation='tanh'))(x)
# x1 = Reshape((1024,))(x)

x = Dense(512)(x1)
x = LeakyReLU(alpha=0.01)(x)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.01)(x)
x = Dense(NB_GENRES)(x)
genre_output = Softmax(name='style_output')(x)

# genre_model = Model(drum_input, genre_output)

In [6]:
## pre-trained genre classification model
#genre_model.load_weights("./tmp/rythm_classification_gm_12-0.30.hdf5")

In [7]:
discriminator_c = Model(drum_input, [output, genre_output])
discriminator_c.summary()

optimizer = Adam() # higher leraning rate for D
discriminator_c.compile(optimizer=optimizer, loss={'gan_output':'binary_crossentropy','style_output':'categorical_crossentropy'},
                      loss_weights=[1.0, 0.5],
                    metrics={'gan_output':'binary_accuracy', 'style_output':'categorical_accuracy'})

W0826 14:42:40.910964 140360265770368 deprecation_wrapper.py:119] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 14:42:40.916881 140360265770368 deprecation_wrapper.py:119] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0826 14:42:40.921581 140360265770368 deprecation.py:323] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/tensorflow/python/ops/nn_impl.py:180: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
drum_input (InputLayer)         (None, 32, 9)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 32, 128)      37888       drum_input[0][0]                 
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 32, 256)      263168      bidirectional_1[0][0]            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 32, 32)       34944       bidirectional_2[0][0]            
__________________________________________________________________________________________________
reshape_1 

In [8]:
discriminator_a = Model(drum_input, output)
discriminator_a.summary()

optimizer = Adam() # higher leraning rate for D
discriminator_a.compile(optimizer=optimizer, 
                        loss={'gan_output':'binary_crossentropy'},
                    metrics={'gan_output':'binary_accuracy'})

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
drum_input (InputLayer)      (None, 32, 9)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           37888     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 256)           263168    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32, 32)            34944     
_________________________________________________________________
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
__________

In [9]:
print discriminator_c.metrics_names 
print discriminator_a.metrics_names

['loss', 'gan_output_loss', 'style_output_loss', 'gan_output_binary_accuracy', 'style_output_categorical_accuracy']
['loss', 'binary_accuracy']


## GENERATOR 

In [10]:
from keras.layers import Reshape, Conv2DTranspose, RepeatVector, Activation,Bidirectional,multiply
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D

len_input = 100

z_input = Input(shape=(len_input,))  # tensorflow order

# # Conditioning label
# label_input = Input(shape=(1,), dtype='int32')
# label_embedding = Embedding(NB_GENRES, len_input, input_length=1)(label_input)
# label_embedding = Flatten()(label_embedding)

# # Element-wise product of the vectors z and the label embeddings
# inputs2 = multiply([z_input, label_embedding])

# 
x = Dense(512)(z_input)
x = LeakyReLU(alpha=0.2)(x)
#x = BatchNormalization(momentum=0.9)(x)
# x = Dense(2048)(x)
# x = LeakyReLU(alpha=0.2)(x)
#x = BatchNormalization(momentum=0.9)(x)
x = Reshape((32, 16))(x)
x = Dropout(dropout_rate)(x)

x = LSTM(256, return_sequences=True, activation='tanh')(x) 
x = LSTM(256, return_sequences=True, activation='tanh')(x)
note_out = LSTM(9, return_sequences=True, activation='relu')(x)
#note_out = LeakyReLU()(x)

# # decoder for note
# x = Dense(64, activation='relu')(z_input)
# x = Dropout(dropout_rate)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(dropout_rate)(x)
# x1 = Dense(512, activation='relu')(x)
# x = Dropout(dropout_rate)(x1)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(dropout_rate)(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(dropout_rate)(x)
# x = Dense(288, activation='sigmoid')(x) # output range shoud be 0. - 1.
# note_out = Reshape((len_seq, nb_notes), name='note_output')(x)

generator = Model(z_input, note_out)
generator.summary()

W0826 14:42:41.994599 140360265770368 deprecation_wrapper.py:119] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0826 14:42:42.002454 140360265770368 deprecation.py:506] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 32, 16)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 16)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32, 256)           279552    
_________________________________________________________________
lstm_5 (LSTM)                (None, 32, 256)           525312    
__________

## ADVERSARIAL MODEL 

In [11]:
def style_ambiguity_loss(y_true, y_pred):
    even_dist = K.ones_like(y_pred) * 1.0/float(len(GENRES))
    return K.categorical_crossentropy(even_dist, y_pred, from_logits=True)

In [12]:

from keras.models import Sequential


# define input for the combined GAN model
z_input = Input(shape=(len_input,))  # tensorflow order
img_gan = generator(z_input)

# training is disable for discriminator in adversarial model
discriminator_c.trainable = False 

# define output
prediction_gan = discriminator_c(img_gan)
print(prediction_gan)

# define combined GAN model
gan = Model(z_input, prediction_gan)
optimizer = Adam()
gan.compile(optimizer=optimizer, loss=['binary_crossentropy', style_ambiguity_loss], 
            loss_weights=[1.0, 0.5], metrics=['binary_accuracy'])
gan.summary()


W0826 14:42:44.665159 140360265770368 deprecation.py:323] From /home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:3298: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



[<tf.Tensor 'model_1/gan_output/Sigmoid:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'model_1/style_output/Softmax:0' shape=(?, 7) dtype=float32>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_3 (Model)              (None, 32, 9)             866152    
_________________________________________________________________
model_1 (Model)              [(None, 1), (None, 7)]    1650312   
Total params: 2,516,464
Trainable params: 866,152
Non-trainable params: 1,650,312
_________________________________________________________________


In [13]:
gan.metrics_names

['loss',
 'model_1_loss',
 'model_1_loss',
 'model_1_binary_accuracy',
 'model_1_binary_accuracy_1']

## Training

In [14]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# Create Z for generator
def get_noise(batch_size, len_input):
#    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, len_input])
    
    # better to use a spherical Z. according to https://github.com/soumith/ganhacks
    noise = np.random.normal(0.0, 0.50, size=[batch_size, len_input])
    return noise

def plot_drum_matrix(a):
    if a is not None:
        a = np.transpose(np.squeeze(a))
        plt.matshow(a)
        plt.show()  

In [15]:
from tensorboard_logger import configure, log_value

from datetime import datetime
now = datetime.now()
logdir = "/tmp/tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"
cmd = "tensorboard --logdir=" + logdir
print cmd

configure(logdir, flush_secs=5)


tensorboard --logdir=/tmp/tf_logs/20190826-144245/


In [ ]:
from keras.utils import to_categorical 

batch_size = 32
nb_epochs = 500
nb_samples = matrices_drums.shape[0]

K_unrolled = 1
MAX_LOSS_RATIO = 3.0

# Labels for real drums: all ones
real_labels = np.ones((batch_size, 1)) # one-sided soft labeling for better stability

# Labels for fake drums: all zeros
fake_labels = np.zeros((batch_size, 1))

train_d = True
train_g = True

for epoch in range(0, nb_epochs):
    nb_steps = int(nb_samples/batch_size)
    
    for repeat in range(nb_steps):
        step = nb_steps * epoch + repeat
        
        if train_d:
            
            m_d_loss = 0.0
            m_d_accuracy = 0.0
            m_d_accuracy_fake, m_d_accuracy_real = 0.0, 0.0
            m_d_loss_cat = 0.0
            m_d_cat_accuracy = 0.0
            
            for j in range(K_unrolled):
                # training data
                random_indices = np.random.randint(0, matrices_drums.shape[0], size=batch_size)
                drum_train = matrices_drums[random_indices, :, :]
                labels_traing = matrices_genres[random_indices]
                labels_traing_categorical = to_categorical(labels_traing, num_classes=NB_GENRES)

                # generated samples
                noise = get_noise(batch_size, len_input)
                drum_fake = generator.predict(noise)

                # training D
                _, d_loss_real, d_loss_cat, d_acc_real, cat_accuracy = discriminator_c.train_on_batch(drum_train, [real_labels, labels_traing_categorical]) 
                d_loss_fake, d_acc_fake = discriminator_a.train_on_batch(drum_fake, fake_labels) 
    
                m_d_loss += 0.5 * (d_loss_real + d_loss_fake)
                m_d_loss_cat += d_loss_cat
                m_d_accuracy +=  0.5 * (d_acc_real + d_acc_fake)
                m_d_accuracy_fake += d_acc_fake
                m_d_accuracy_real += d_acc_real
                m_d_cat_accuracy += cat_accuracy
                # cache for later update
    #             cache_weights = discriminator.get_weights()
            
            m_d_loss /= float(K_unrolled)
            m_d_accuracy /= float(K_unrolled)
            m_d_accuracy_fake /= float(K_unrolled)
            m_d_accuracy_real /= float(K_unrolled)
            m_d_loss_cat /= float(K_unrolled)
            m_d_cat_accuracy /= float(K_unrolled)
           
            # store value
            log_value("D loss", m_d_loss, step)  
            log_value("D accuracy", m_d_accuracy, step)  
            log_value("D accuracy - fake", m_d_accuracy_fake, step) 
            log_value("D accuracy - real", m_d_accuracy_real, step) 
            
            log_value("D category loss", m_d_loss_cat, step)  
            log_value("D category accuracy", m_d_cat_accuracy, step) 
            log_value("D total loss", m_d_loss + m_d_loss_cat, step)
            
        # training G
        if train_g:
            y = np.ones([batch_size, 1]) # watch out the label! it should be one here        

            noise = get_noise(batch_size, len_input)

            # Get a batch of random labels
            labels_random = np.random.randint(0, NB_GENRES, batch_size).reshape(-1, 1)

            _, m_a_loss, m_a_cat_loss, m_a_accuracy, _ = gan.train_on_batch(noise, [y, labels_random])
            
            # store value
            log_value("G loss", m_a_loss, step)
            log_value("G accuracy", m_a_accuracy, step)
            log_value("style amguity loss", m_a_cat_loss, step)
            log_value("G total loss", m_a_loss + m_a_cat_loss, step)
        
        if train_d and train_g:
            if m_a_loss / m_d_loss > MAX_LOSS_RATIO:
                train_d = False
                print "Pausing D"
            elif m_d_loss / m_a_loss > MAX_LOSS_RATIO:
                train_g = False
                print "Pausing G"
        else:
            train_d = True
            train_g = True
            
        # update layer 
#         discriminator.set_weights(cache_weights)
 
        if repeat % 100 == 0:
            
            print("epoch", epoch, repeat)
            print("d_loss", m_d_loss, "d_cat_acc", m_d_cat_accuracy,
                "a_loss", m_a_loss, 'a_cat_loss', m_a_cat_loss) # print mean loss)
            print("d_accuracy", m_d_accuracy )
            
            # sample output
            noise = get_noise(1, len_input)
            drum_generated = generator.predict(noise)           
            plot_drum_matrix(drum_generated)
            
            print
            print
            
                        
    # store temporary models
    if (epoch + 1) % 5 == 0:
        generator.save("./tmp/can3_generator-epoch-%03d-%0.5f.h5" % (epoch, m_a_loss))
        gan.save("./tmp/can3_gan-epoch-%03d-%0.5f.h5" % (epoch, m_a_loss))
        discriminator_c.save("./tmp/can3_discriminator_c-epoch-%03d-%0.5f.h5" % (epoch, m_d_loss))
        discriminator_a.save("./tmp/can3_discriminator_a-epoch-%03d-%0.5f.h5" % (epoch, m_d_loss))

/home/nao/anaconda3/envs/p2/lib/python2.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [ ]:
generator.save("./tmp/can_generator.h5")
gan.save("./tmp/can_gan.h5")
discriminator.save("./tmp/can_discriminator.h5")

In [ ]:
for i in range(3):
    noise = get_noise(1, len_input)
    drum_generated = generator.predict(noise)
    plot_drum_matrix(drum_generated)

# MIDI Playback

In [ ]:
# midi playback
def note_matrix_to_sequence(mat, threshold = 0.5):
    seq = []
    for row in mat[:]:
        arow = [[i, r] for i, r in enumerate(row) if r > threshold]
        seq.append(arow)
    return seq

import OSC
max_poly = 6 # maximum number of instruments played at the same time

client = OSC.OSCClient()
client.connect( ('10.0.1.14', 2014) ) 

def send_sequence_via_osc(seq):
    ## the most basic ##
    msg = OSC.OSCMessage()
    msg.setAddress("/seq")
    msg.append(max_poly * 2)
    
    for notes in seq:
        for i in range(max_poly):
            if len(notes) > i:
                msg.append([notes[i][0]+min_drum_note, notes[i][1]])
            else:
                msg.append([0, 0])
    client.send(msg)
    
def playback_seq_via_osc(mat):
    send_sequence_via_osc(note_matrix_to_sequence(mat, 0.2))

In [ ]:
import time

repeat = 1

for j in range(100):
    noise1 = get_noise(1, len_input) 
    noise2 = get_noise(1, len_input) 
    
    for i in range(repeat):
        noise = noise1 * (1.0 - i/float(repeat)) + noise2 * i/float(repeat)
        drum_generated = generator.predict(noise)
        mat = np.squeeze(drum_generated)
        playback_seq_via_osc(mat)
        time.sleep(4.0)
        clear_output(wait=True)